In [ ]:
!git clone https://github.com/komazawa-katolab2023/gpt_fastapi.git

Cloning into 'gpt_fastapi'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 13 (delta 1), reused 12 (delta 0), pack-reused 0
Receiving objects: 100% (13/13), 35.43 KiB | 625.00 KiB/s, done.
Resolving deltas: 100% (1/1), done.


In [ ]:
%cd gpt_fastapi

/content/gpt_fastapi


In [ ]:
!pip install fastapi nest-asyncio uvicorn pyngrok openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

ngrokを使って、Colabで立ち上げたWebアプリにアクセスします。
ngrokは、ローカルPC上で稼働しているネットワークサービスを外部公開（デプロイ）できるサービスです。
これを使うことで、ローカルで立ち上げたWebサーバーを外部に公開することができます。

ngrokのトークン取得
https://zenn.dev/protoout/articles/47-ngrok-setup-2022

In [ ]:
import nest_asyncio
from pyngrok import ngrok

ngrok.set_auth_token("") #取得したトークンをここに入力

In [ ]:
from fastapi import FastAPI, Form
from fastapi.responses import HTMLResponse, JSONResponse
import os
import openai
from openai import OpenAI
import uvicorn

app = FastAPI()

# APIキーとモデル名を設定
API_KEY = "" # sk-から始まるOpenAIのAPIキーを入れる

client = OpenAI(api_key=API_KEY)

# 会話履歴を保持するリスト
conversation_history = []

@app.get("/", response_class=HTMLResponse)
async def index():
    with open("templates/index.html", "r") as file:
        return HTMLResponse(content=file.read(), status_code=200)

@app.post("/ask")
async def ask(user_input: str = Form(...), gpt_model: str = Form(...)):
    system_prompt = "あなたはAIアシスタントです。"
    response = client.chat.completions.create(
    model=gpt_model,
    messages=[
        {"role": "system","content":system_prompt},
        {"role": "user", "content":user_input},
    ]
    )
    result = response.choices[0].message.content

    # 会話履歴に追加
    conversation_history.append({'user': user_input, 'bot': result})

    return JSONResponse(content={'response': result})

ngrok_tunnel=ngrok.connect(8000)
print('PUBLIC_URL:',ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, host='127.0.0.1', port=8000)

PUBLIC_URL: https://da1c-34-125-229-189.ngrok-free.app


INFO:     Started server process [919]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2409:10:900:9f00:789b:5168:d2d7:3b05:0 - "GET / HTTP/1.1" 200 OK
INFO:     2409:10:900:9f00:789b:5168:d2d7:3b05:0 - "GET /history HTTP/1.1" 404 Not Found
INFO:     2409:10:900:9f00:789b:5168:d2d7:3b05:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [919]


KeyboardInterrupt: 